In [39]:
import pandas as pd

import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, GroupKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier


In [40]:
data = pd.read_pickle('../cytofdata/pool1_with_annotated_clusters')
data.drop('batch', axis=1, inplace=True)

data['sample'] = data['patient_number'].astype(str) + "_" + data['test_time'].astype(str)
data

,89Y_CD45,111Cd_CD3,112Cd_CD34,113Cd_CD123,114Cd_CD66b,116Cd_HLA-DR,141Pr_CD38,142Nd_cCaspase3,143Nd_pCRKL Y207,144Nd_pTyr,...,173Yb_STAT3tot,174Yb_CD11c,175Lu_CXCR4,176Yb_pS6 S240244,195Pt_mBC2,209Bi_CD11b,patient_number,test_time,cluster,sample
0,2.217051,0.113545,3.317439,0.381279,1.590146,2.885046,3.629630,0.186837,0.327056,0.842817,...,2.558300,0.000000,0.953744,0.000000,3.512650,0.531857,01,0,HSPCs,01_0
1,3.963797,0.000000,0.311732,0.000000,4.716617,0.714334,0.808533,0.152932,0.676452,0.183566,...,2.994078,1.007841,1.161596,0.040417,3.752279,3.391960,01,0,Neutrophils,01_0
2,2.961751,0.231498,0.041203,0.275047,4.226237,0.250043,0.766132,0.000000,0.075735,1.489841,...,3.148486,0.135580,0.000000,0.749384,3.559952,2.233353,01,0,Neutrophils,01_0
3,3.951971,0.391544,0.030235,1.911737,1.914248,0.196068,4.125344,0.238149,0.496188,0.466622,...,2.985532,0.909775,2.843206,1.296746,3.968642,0.490613,01,0,Basophils,01_0
4,3.204973,0.000000,0.002475,0.400668,4.130337,0.454735,0.448444,0.079543,0.328764,1.208815,...,3.489844,0.758197,0.813311,0.877981,2.142285,2.659254,01,0,Neutrophils,01_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26288919,1.693280,0.348215,0.502324,0.185988,3.035682,0.350522,2.593030,0.211198,1.259692,0.766676,...,4.092850,0.000000,0.000000,3.178231,4.793082,0.000000,HDBM_532,None,NeutrophilCD16neg,HDBM_532_None
26520713,0.729598,0.485712,0.172502,0.000000,0.772020,0.000000,0.719245,0.417552,1.475494,1.154216,...,3.757267,0.099147,0.294973,0.936303,4.438051,0.384407,HDBM_630,None,NeutrophilCD16neg,HDBM_630_None
26031470,2.254079,0.862573,0.441509,0.498153,3.449804,0.328851,1.732715,0.144498,0.869975,1.309697,...,4.134409,0.090517,0.929077,1.499913,5.032999,1.446886,HDBM_436,None,Neutrophils,HDBM_436_None
27498207,4.621356,3.484828,0.077849,0.097483,0.373927,0.000000,0.195959,0.000000,0.244078,0.420155,...,1.483905,0.000000,1.437943,0.186262,3.698609,0.000000,HDPB_625,None,Th,HDPB_625_None


In [41]:
data['cluster'].unique()

array(['HSPCs', 'Neutrophils', 'Basophils', 'Monocytes', 'Eosinophils',
       'NeutrophilCD16neg', 'Tc', 'Th', 'NKcells', 'PreNeu', 'PreNeu2',
       'Bcells', 'PreNeu1', 'pDC', 'Doublets', 'PreNeu3', 'LinNeg',
       'NeutrophilsHLA', 'NeutrophilsCD25'], dtype=object)

In [42]:
cluster_as_num = {
    'PreNeu1'            :18,
    'pDC'                :17,
    'PreNeu3'            :16,
    'NeutrophilsCD25'    :15,
    'PreNeu2'            :14,
    'NeutrophilsHLA'     :13,
    'LinNeg'             :12,
    'Doublets'           :11,
    'Bcells'             :10,
    'HSPCs'              :9,
    'NKcells'            :8,
    'Tc'                 :7,
    'Th'                 :6,
    'Monocytes'          :5,
    'Basophils'          :4,
    'Eosinophils'        :3,
    'NeutrophilCD16neg'  :2 ,
     'PreNeu'            :1 ,
     'Neutrophils'       :0 
}

data['cluster'] = data['cluster'].map(cluster_as_num)

In [43]:
data

,89Y_CD45,111Cd_CD3,112Cd_CD34,113Cd_CD123,114Cd_CD66b,116Cd_HLA-DR,141Pr_CD38,142Nd_cCaspase3,143Nd_pCRKL Y207,144Nd_pTyr,...,173Yb_STAT3tot,174Yb_CD11c,175Lu_CXCR4,176Yb_pS6 S240244,195Pt_mBC2,209Bi_CD11b,patient_number,test_time,cluster,sample
0,2.217051,0.113545,3.317439,0.381279,1.590146,2.885046,3.629630,0.186837,0.327056,0.842817,...,2.558300,0.000000,0.953744,0.000000,3.512650,0.531857,01,0,9,01_0
1,3.963797,0.000000,0.311732,0.000000,4.716617,0.714334,0.808533,0.152932,0.676452,0.183566,...,2.994078,1.007841,1.161596,0.040417,3.752279,3.391960,01,0,0,01_0
2,2.961751,0.231498,0.041203,0.275047,4.226237,0.250043,0.766132,0.000000,0.075735,1.489841,...,3.148486,0.135580,0.000000,0.749384,3.559952,2.233353,01,0,0,01_0
3,3.951971,0.391544,0.030235,1.911737,1.914248,0.196068,4.125344,0.238149,0.496188,0.466622,...,2.985532,0.909775,2.843206,1.296746,3.968642,0.490613,01,0,4,01_0
4,3.204973,0.000000,0.002475,0.400668,4.130337,0.454735,0.448444,0.079543,0.328764,1.208815,...,3.489844,0.758197,0.813311,0.877981,2.142285,2.659254,01,0,0,01_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26288919,1.693280,0.348215,0.502324,0.185988,3.035682,0.350522,2.593030,0.211198,1.259692,0.766676,...,4.092850,0.000000,0.000000,3.178231,4.793082,0.000000,HDBM_532,None,2,HDBM_532_None
26520713,0.729598,0.485712,0.172502,0.000000,0.772020,0.000000,0.719245,0.417552,1.475494,1.154216,...,3.757267,0.099147,0.294973,0.936303,4.438051,0.384407,HDBM_630,None,2,HDBM_630_None
26031470,2.254079,0.862573,0.441509,0.498153,3.449804,0.328851,1.732715,0.144498,0.869975,1.309697,...,4.134409,0.090517,0.929077,1.499913,5.032999,1.446886,HDBM_436,None,0,HDBM_436_None
27498207,4.621356,3.484828,0.077849,0.097483,0.373927,0.000000,0.195959,0.000000,0.244078,0.420155,...,1.483905,0.000000,1.437943,0.186262,3.698609,0.000000,HDPB_625,None,6,HDPB_625_None


In [44]:

internal_and_external = ['89Y_CD45', 
                '111Cd_CD3', 
                '112Cd_CD34',
                '113Cd_CD123', 
                '114Cd_CD66b', 
                '116Cd_HLA-DR', 
                '141Pr_CD38', 
                '142Nd_cCaspase3', 
                '143Nd_pCRKL Y207', 
                '144Nd_pTyr',
                '145Nd_CD4', 
                '146Nd_CD49d', 
                '147Sm_CD20', 
                '148Nd_CD16', 
                '149Sm_CD25',
                '150Nd_pSTAT5 Y694', 
                '151Eu_pSTAT3 S727', 
                '152Sm_CD13',
                '153Eu_pSTAT1 Y701', 
                '154Sm_CD45RA', 
                '155Gd_CD27',
                '156Gd_pp38 T180Y182', 
                '157Gd_CD8a', 
                '158Gd_pSTAT3 Y705',
                '159Tb_pMAPKAPK T334', 
                '160Gd_CD14', 
                '161Dy_CD26', 
                '162Dy_FoxP3',
                '163Dy_CD56', 
                '164Dy_CD15', 
                '165Ho_pCREB S133', 
                '166Er_MPO',
                '167Er_IL1-RAP', 
                '168Er_CD117', 
                '169Tm_CD33', 
                '170Er_pSRC Y418',
                '171Yb_pERK T202Y204', 
                '172Yb_pS6 S235S236', 
                '173Yb_STAT3tot',
                '174Yb_CD11c', 
                '175Lu_CXCR4', 
                '176Yb_pS6 S240244', 
                '195Pt_mBC2', 
                '209Bi_CD11b']    
        

external = ['89Y_CD45', 
                '111Cd_CD3', 
                '112Cd_CD34',
                '113Cd_CD123', 
                '114Cd_CD66b', 
                '116Cd_HLA-DR', 
                '141Pr_CD38', 
                '145Nd_CD4', 
                '146Nd_CD49d', 
                '147Sm_CD20', 
                '148Nd_CD16', 
                '149Sm_CD25',
                '152Sm_CD13',
                '154Sm_CD45RA', 
                '155Gd_CD27',
                '157Gd_CD8a', 
                '160Gd_CD14', 
                '161Dy_CD26', 
                '162Dy_FoxP3',
                '163Dy_CD56', 
                '164Dy_CD15', 
                '166Er_MPO',
                '167Er_IL1-RAP', 
                '168Er_CD117', 
                '169Tm_CD33', 
                '174Yb_CD11c', 
                '175Lu_CXCR4', 
                '195Pt_mBC2', 
                '209Bi_CD11b']

internal = list(set(internal_and_external) - set(external))

In [45]:
X = data[external]
y = data['cluster']
groups = data['sample']

In [ ]:
model = LinearDiscriminantAnalysis()

param_grid = [
    {'solver': ['svd'], 'shrinkage': [None]},  
    {'solver': ['lsqr', 'eigen'], 'shrinkage': [None, 'auto', 0.1, 0.5, 0.9]}
]


outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=3)

results = []

for train_idx, test_idx in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    groups_train = groups.iloc[train_idx]

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=inner_cv,
        scoring='accuracy'
    )

    grid_search.fit(X_train, y_train, groups=groups_train)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    f1_scores = f1_score(y_test, predictions, average=None)
    median_f1 = np.median(f1_scores)

    

    true_frequencies = np.bincount(y_test) / len(y_test)
    predicted_frequencies = np.bincount(predictions, minlength=len(np.unique(y))) / len(predictions)
    max_diff = np.max(np.abs(true_frequencies - predicted_frequencies))
    rsse = np.sqrt(np.sum((true_frequencies - predicted_frequencies) ** 2))

    results.append({
        'accuracy': accuracy,
        'median_f1': median_f1,
        'max_diff': max_diff,
        'rsse': rsse,
        'est': grid_search.best_score_,
        'cfg': grid_search.best_params_
    })

    print('One round completed')

results_df = pd.DataFrame(results)
print(results_df)


One round completed
One round completed
One round completed
One round completed
One round completed
   accuracy  median_f1  max_diff      rsse       est  \
0  0.847269   0.818825  0.031361  0.037088  0.861587   
1  0.852001   0.818077  0.023391  0.030414  0.861839   
2  0.877614   0.851668  0.020094  0.025817  0.857399   
3  0.879104   0.793562  0.009104  0.012259  0.856130   
4  0.846435   0.819311  0.053775  0.068130  0.863747   

                                    cfg  
0  {'shrinkage': 0.5, 'solver': 'lsqr'}  
1  {'shrinkage': 0.5, 'solver': 'lsqr'}  
2  {'shrinkage': 0.5, 'solver': 'lsqr'}  
3  {'shrinkage': 0.5, 'solver': 'lsqr'}  
4  {'shrinkage': 0.5, 'solver': 'lsqr'}  


In [ ]:


model_1 = RandomForestClassifier(n_estimators=100)

outer_cv = GroupKFold(n_splits=5)

results_1 = []

for train_idx, test_idx in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    groups_train = groups.iloc[train_idx]

    model_1.fit(X_train, y_train)
    predictions = model_1.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    f1_scores = f1_score(y_test, predictions, average=None)
    median_f1 = np.median(f1_scores)

    true_frequencies = np.bincount(y_test) / len(y_test)
    predicted_frequencies = np.bincount(predictions, minlength=len(np.unique(y))) / len(predictions)
    max_diff = np.max(np.abs(true_frequencies - predicted_frequencies))
    rsse = np.sqrt(np.sum((true_frequencies - predicted_frequencies) ** 2))

    results_1.append({
        'accuracy': accuracy,
        'median_f1': median_f1,
        'max_diff': max_diff,
        'rsse': rsse
    })

    print('One round completed')

# Print results
results_df_rfc = pd.DataFrame(results_1)
print(results_df_rfc)

In [46]:
data

,89Y_CD45,111Cd_CD3,112Cd_CD34,113Cd_CD123,114Cd_CD66b,116Cd_HLA-DR,141Pr_CD38,142Nd_cCaspase3,143Nd_pCRKL Y207,144Nd_pTyr,...,173Yb_STAT3tot,174Yb_CD11c,175Lu_CXCR4,176Yb_pS6 S240244,195Pt_mBC2,209Bi_CD11b,patient_number,test_time,cluster,sample
0,2.217051,0.113545,3.317439,0.381279,1.590146,2.885046,3.629630,0.186837,0.327056,0.842817,...,2.558300,0.000000,0.953744,0.000000,3.512650,0.531857,01,0,9,01_0
1,3.963797,0.000000,0.311732,0.000000,4.716617,0.714334,0.808533,0.152932,0.676452,0.183566,...,2.994078,1.007841,1.161596,0.040417,3.752279,3.391960,01,0,0,01_0
2,2.961751,0.231498,0.041203,0.275047,4.226237,0.250043,0.766132,0.000000,0.075735,1.489841,...,3.148486,0.135580,0.000000,0.749384,3.559952,2.233353,01,0,0,01_0
3,3.951971,0.391544,0.030235,1.911737,1.914248,0.196068,4.125344,0.238149,0.496188,0.466622,...,2.985532,0.909775,2.843206,1.296746,3.968642,0.490613,01,0,4,01_0
4,3.204973,0.000000,0.002475,0.400668,4.130337,0.454735,0.448444,0.079543,0.328764,1.208815,...,3.489844,0.758197,0.813311,0.877981,2.142285,2.659254,01,0,0,01_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26288919,1.693280,0.348215,0.502324,0.185988,3.035682,0.350522,2.593030,0.211198,1.259692,0.766676,...,4.092850,0.000000,0.000000,3.178231,4.793082,0.000000,HDBM_532,None,2,HDBM_532_None
26520713,0.729598,0.485712,0.172502,0.000000,0.772020,0.000000,0.719245,0.417552,1.475494,1.154216,...,3.757267,0.099147,0.294973,0.936303,4.438051,0.384407,HDBM_630,None,2,HDBM_630_None
26031470,2.254079,0.862573,0.441509,0.498153,3.449804,0.328851,1.732715,0.144498,0.869975,1.309697,...,4.134409,0.090517,0.929077,1.499913,5.032999,1.446886,HDBM_436,None,0,HDBM_436_None
27498207,4.621356,3.484828,0.077849,0.097483,0.373927,0.000000,0.195959,0.000000,0.244078,0.420155,...,1.483905,0.000000,1.437943,0.186262,3.698609,0.000000,HDPB_625,None,6,HDPB_625_None


In [47]:
data['sample'].unique()

array(['01_0', '02_0', '03_0', '04_0', '05_0', '06_0', '07_0', '08_0',
       '09_0', '11_0', '12_0', '13_0', '14_0', '15_0', '16_0', '17_0',
       '18_0', '19_0', '20_0', '21_0', '22_0', '23_0', '24_0', '25_0',
       '26_0', '27_0', '28_0', '29_0', '30_0', '31_0', '32_0', '33_0',
       '34_0', '35_0', '36_0', '37_0', '38_0', '43_0', 'HDBM_578_None',
       'HDBM_663_None', 'HDBM_630_None', 'HDBM_436_None', 'HDPB_522_None',
       'HDPB_433_None', 'HDBM_446_None', 'HDPB_656_None', 'HDBM_532_None',
       'HDPB_424_None', 'HDPB_597_None', 'HDPB_625_None', '01_1', '01_2',
       '02_1', '02_2', '03_1', '03_2', '04_1', '04_2', '05_1', '05_2',
       '06_1', '06_2', '07_1', '07_2', '08_1', '08_2', '09_1', '09_2',
       '11_1', '11_2', '12_1', '12_2', '13_1', '13_2', '14_1', '14_2',
       '15_1', '15_2', '16_1', '16_2', '17_1', '17_2', '18_1', '18_2',
       '19_1', '19_2', '20_1', '20_2', '21_1', '21_2', '22_1', '22_2',
       '23_1', '23_2', '24_1', '24_2', '25_1', '25_2', '26_1', '2

In [48]:
data_test = data[data['sample'].isin(['01_0', '01_1', '01_2',
                                   '12_0', '12_1', '12_2',
                                   '08_0', '08_1', '08_2',
                                   '15_0', '15_1', '15_2',
                                   '20_0', '20_1', '20_2',
                                   '28_0', '28_1', '28_2',
                                   '35_0', '35_1', '35_2',
                                   'HDBM_663_None', 'HDPB_597_None', 'HDBM_578_None',])]

data_train = data[~data['sample'].isin(['01_0', '01_1', '01_2',
                                   '12_0', '12_1', '12_2',
                                   '08_0', '08_1', '08_2',
                                   '15_0', '15_1', '15_2',
                                   '20_0', '20_1', '20_2',
                                   '28_0', '28_1', '28_2',
                                   '35_0', '35_1', '35_2',
                                   'HDBM_663_None', 'HDPB_597_None', 'HDBM_578_None',])]



In [74]:
data_train


,89Y_CD45,111Cd_CD3,112Cd_CD34,113Cd_CD123,114Cd_CD66b,116Cd_HLA-DR,141Pr_CD38,142Nd_cCaspase3,143Nd_pCRKL Y207,144Nd_pTyr,...,173Yb_STAT3tot,174Yb_CD11c,175Lu_CXCR4,176Yb_pS6 S240244,195Pt_mBC2,209Bi_CD11b,patient_number,test_time,cluster,sample
560879,1.670344,0.000000e+00,0.000000,0.000000e+00,1.619497,0.087057,1.885950,0.000000,0.177320,0.000000,...,2.913760,0.000000,0.700987,0.155629,3.346074,0.259070,02,0,2,02_0
560880,3.539977,2.142803e-01,0.134097,1.160107e-02,3.746725,0.000000,0.000000,0.000000,0.446203,0.758547,...,2.572958,2.718444,0.661870,1.697639,2.850482,2.801288,02,0,0,02_0
560881,1.885515,1.464498e-10,0.000000,3.574960e-10,2.373047,0.000000,0.084453,0.069303,0.000000,1.342216,...,3.339220,1.635234,0.974204,0.907949,3.483692,1.487358,02,0,0,02_0
560882,1.611550,6.270524e-01,0.000000,0.000000e+00,3.943385,0.420081,0.390100,0.000000,0.638499,1.095954,...,3.530083,0.000000,0.548274,1.144781,3.580094,1.220637,02,0,0,02_0
560883,1.901668,0.000000e+00,0.000000,2.606432e-01,2.794307,0.000000,0.218226,0.000000,0.431534,1.165762,...,3.511482,0.000000,1.435172,0.252553,3.602126,1.612003,02,0,0,02_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26029571,1.913194,6.111502e-01,0.396121,1.461382e-01,3.280591,0.000000,0.508238,0.314672,1.867492,0.875840,...,3.776036,0.000000,0.657415,1.903500,4.993514,0.942108,HDBM_436,None,0,HDBM_436_None
26288919,1.693280,3.482150e-01,0.502324,1.859879e-01,3.035682,0.350522,2.593030,0.211198,1.259692,0.766676,...,4.092850,0.000000,0.000000,3.178231,4.793082,0.000000,HDBM_532,None,2,HDBM_532_None
26520713,0.729598,4.857115e-01,0.172502,0.000000e+00,0.772020,0.000000,0.719245,0.417552,1.475494,1.154216,...,3.757267,0.099147,0.294973,0.936303,4.438051,0.384407,HDBM_630,None,2,HDBM_630_None
26031470,2.254079,8.625727e-01,0.441509,4.981533e-01,3.449804,0.328851,1.732715,0.144498,0.869975,1.309697,...,4.134409,0.090517,0.929077,1.499913,5.032999,1.446886,HDBM_436,None,0,HDBM_436_None


In [75]:
data_train['cluster'].value_counts()

cluster
0     7719990
2     5244406
1     1430020
3      448990
4      357445
5      323213
6      141641
7      104153
8       80176
9       44503
12      39523
10      38636
13      36411
14      20014
16      17297
11      14493
15       8391
18       5056
17       2526
Name: count, dtype: int64

In [76]:
data_test

,89Y_CD45,111Cd_CD3,112Cd_CD34,113Cd_CD123,114Cd_CD66b,116Cd_HLA-DR,141Pr_CD38,142Nd_cCaspase3,143Nd_pCRKL Y207,144Nd_pTyr,...,173Yb_STAT3tot,174Yb_CD11c,175Lu_CXCR4,176Yb_pS6 S240244,195Pt_mBC2,209Bi_CD11b,patient_number,test_time,cluster,sample
0,2.217051,0.113545,3.317439,0.381279,1.590146,2.885046,3.629630,0.186837,0.327056,0.842817,...,2.558300,0.000000,0.953744,0.000000,3.512650,0.531857,01,0,9,01_0
1,3.963797,0.000000,0.311732,0.000000,4.716617,0.714334,0.808533,0.152932,0.676452,0.183566,...,2.994078,1.007841,1.161596,0.040417,3.752279,3.391960,01,0,0,01_0
2,2.961751,0.231498,0.041203,0.275047,4.226237,0.250043,0.766132,0.000000,0.075735,1.489841,...,3.148486,0.135580,0.000000,0.749384,3.559952,2.233353,01,0,0,01_0
3,3.951971,0.391544,0.030235,1.911737,1.914248,0.196068,4.125344,0.238149,0.496188,0.466622,...,2.985532,0.909775,2.843206,1.296746,3.968642,0.490613,01,0,4,01_0
4,3.204973,0.000000,0.002475,0.400668,4.130337,0.454735,0.448444,0.079543,0.328764,1.208815,...,3.489844,0.758197,0.813311,0.877981,2.142285,2.659254,01,0,0,01_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27445403,1.795964,0.317917,0.061669,0.000000,4.030856,0.141028,1.313184,0.373752,1.359254,0.123516,...,3.668734,0.000000,0.915739,1.549029,4.524221,1.790981,HDPB_597,None,0,HDPB_597_None
26434333,1.653278,0.721037,0.000000,0.244350,4.123832,0.000000,1.093431,0.557464,1.389568,0.860432,...,4.102405,0.453451,1.400038,1.406475,4.546818,1.848670,HDBM_578,None,0,HDBM_578_None
26638016,2.495669,0.705190,0.019936,0.746797,0.531714,4.552544,3.624877,0.128357,0.068003,1.405081,...,2.890788,0.254210,2.613148,1.328578,4.285229,0.153277,HDBM_663,None,9,HDBM_663_None
26437597,1.818850,0.080734,1.035536,0.029431,4.158769,0.135923,0.728520,0.126227,1.467396,1.024819,...,3.245868,0.000000,0.716784,1.692657,4.605637,1.406965,HDBM_578,None,0,HDBM_578_None


In [77]:
data_test['cluster'].value_counts()

cluster
2     2139629
0     1448559
1      174906
3      131310
4      126294
5       94363
6       34822
12      30806
7       27749
9       25996
8       14516
10       7318
16       6618
13       6579
14       3492
11       2400
18        697
17        697
15        139
Name: count, dtype: int64

In [49]:
model_1 = RandomForestClassifier(n_estimators=100)

X = data_train[external]
y = data_train['cluster']

model_1.fit(X, y)

RandomForestClassifier()

In [50]:
X_test = data_test[external]
y_test = data_test['cluster']

In [51]:
model_1.score(X_test, y_test)

0.9359875984652399

In [58]:
import plotly.express as px
from sklearn.metrics import confusion_matrix

y_pred = model_1.predict(X_test)

In [64]:
cm = confusion_matrix(y_test, y_pred)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

class_labels = [str(i) for i in range(19)]

In [70]:
import plotly.express as px

fig = px.imshow(
    cm_normalized,
    text_auto=True,
    labels=dict(x="Predicted Class", y="Actual Class", color="Normalized Count"),
    x=class_labels,
    y=class_labels,
    color_continuous_scale='Greens'
)

fig.update_layout(
    title="Normalized Confusion  for the annotated model",
    xaxis_title="Predicted Class",
    yaxis_title="Actual Class"
)

fig.show()


In [73]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_test, y_pred)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Original Confusion Matrix", "Normalized Confusion Matrix"),
    horizontal_spacing=0.15  
)

fig.add_trace(
    go.Heatmap(
        z=cm,
        x=class_labels,
        y=class_labels,
        colorscale='Greens',
        text=cm,
        texttemplate="%{text}",
        showscale=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=cm_normalized,
        x=class_labels,
        y=class_labels,
        colorscale='Greens',
        text=np.round(cm_normalized, 2),
        texttemplate="%{text}",
        showscale=True
    ),
    row=1, col=2
)

fig.update_layout(
    title_text="Confusion for the annotated model",
    xaxis_title="Predicted Class",
    yaxis_title="Actual Class",
    xaxis2_title="Predicted Class",
    yaxis2_title="Actual Class"
)

fig.show()


# Bruker trent modell til å predikere celle typene i pool2 

In [52]:
data_pool_2 = pd.read_pickle('../cytofdata/uncorrected_batch_5_6.pkl')

In [53]:
data_pool_2 = data_pool_2[~data_pool_2['file_id'].isin(['RT', 'RJ'])]

In [54]:
X_pool_2 = data_pool_2[external]
y_pool_2 = model_1.predict(X_pool_2)

data_pool_2['cluster'] = y_pool_2

In [55]:
data_pool_2['cluster'].value_counts()

cluster
0     3650076
2     1930661
5      470793
1      413947
3      247067
4      192916
6      155911
7      116862
8       51528
11      41629
10      29291
14      16073
15      12301
9       11100
16       8641
12       8517
13       2874
17       2101
18       1662
Name: count, dtype: int64

In [56]:
data_pool_2.to_pickle('../cytofdata/data_clusters_batch5_6_annotated_mode.pkl')